<a href="https://colab.research.google.com/github/akshatjain99/KairoGuard/blob/master/indic2019_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from keras import backend as K
from google.colab import drive
drive.mount('/gdrive')
import os
import re
import shutil
import tarfile
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import IPython.display as display
import tensorboardcolab as tbc
tbc = tbc.TensorBoardColab()

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
Wait for 8 seconds...
TensorBoard link:
https://7909fcf6.ngrok.io


In [0]:
writer = tbc.get_writer()

In [0]:
batch_size = 10
nb_boxes=1
grid_w=2
grid_h=2
cell_w=14
cell_h=14
img_w=28
img_h=28
img_channels = 1
input_shape = (batch_size, img_w, img_h, img_channels)
print(input_shape)

(10, 28, 28, 1)


In [0]:
def extract_fn(data_record):
    features = {
      'image': tf.FixedLenFeature([], tf.string),
      'label': tf.FixedLenFeature([5], tf.float32)
    }
    data = tf.parse_single_example(data_record, features)
    
    img1 = tf.decode_raw(data['image'], tf.float32)
    img1 = tf.reshape(img1, (img_w, img_h, img_channels))
    
    
    return img1, data['label']


In [0]:
input_pattern = '/gdrive/My Drive/indic2019/TFRecords/*.tfrecord'

In [0]:
def input_fn(files_pattern,  batch_size, mode=tf.estimator.ModeKeys.TRAIN):
  files = tf.data.Dataset.list_files(files_pattern, shuffle=True)
  dataset = files.apply(tf.contrib.data.parallel_interleave( lambda filename: tf.data.TFRecordDataset(filename), cycle_length=1))
    
  is_training = (mode == tf.estimator.ModeKeys.TRAIN)
  if is_training:
    buffer_size = batch_size * 2 + 1
    dataset = dataset.shuffle(buffer_size=buffer_size)

    # Transformation
  dataset = dataset.map(extract_fn)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(2 * batch_size)

  image, label = dataset.make_one_shot_iterator().get_next()
  features = {'images': image}
  
  
  return features, label

In [0]:
def base_model(input,batch_size, input_shape):
  
  #print(input.shape)
  #print(batch_size)
  
  
  #Conv Layer - 1
  x = tf.keras.layers.Conv2D(16,(5,5),activation='relu', input_shape = input_shape , name="Conv_1",
                             use_bias=True, kernel_initializer='glorot_uniform', 
                             bias_initializer='zeros')(input)
  #x = tf.keras.layers.MaxPooling2D()(x)
  print(x)
  
  #Flatten it out
  x = tf.keras.layers.Flatten(name="Flatten_1")(x)
  print("After flatten")
  print(x)
  
  #Dense layer
  x = tf.keras.layers.Dense(9216, activation="sigmoid", name="Dense1")(x)
  #print(x)
  x = tf.keras.layers.Dense(5, activation='sigmoid')(x)
  print(x)
  # x = tf.keras.layers.Reshape((2*2, (1*5)), name= 'model_final_reshape')(x)
  
  return x

In [0]:
def get_feature_columns():
  feature_columns = {'images': tf.feature_column.numeric_column('images', (28,28))}
  return feature_columns

In [0]:

def custom_loss(labels,logits):
  
  print(logits)
  
  true_confidence = labels[:,0]
  true_x=labels[:,1]
  true_y=labels[:,2]
  true_w=labels[:,3]
  true_h=labels[:,4]

  predict_confidence=logits[:,0]
  predict_x=logits[:,1]
  predict_y=logits[:,2]
  predict_w=logits[:,3]
  predict_h=logits[:,4]	

  xy_loss= K.square(true_x-predict_x) + K.square(true_y-predict_y)
  wh_loss= K.square(K.sqrt(true_w)-K.sqrt(predict_w))+ K.square(K.sqrt(true_h)-K.sqrt(predict_h))

  con_loss=K.square(true_confidence-predict_confidence)

  loss= xy_loss + wh_loss + con_loss
  return tf.math.reduce_mean(loss)

In [0]:
def model_fn(features, labels, mode, params):
  
  print(mode)
  # Create the input layers from the features                                                                                               
  feature_columns = list(get_feature_columns().values())
  images = tf.feature_column.input_layer(features=features, feature_columns=feature_columns)
  print('pre',images.shape)
  images = tf.reshape(images, shape=(-1, 28, 28, 1),name='my_reshape')
  print('post',images.shape)
  print(images)

  # batch_size = 300
  # Calculate logits through CNN                                                                                                            
  logits = base_model(images,batch_size, input_shape)
  
  print('Printing Logits')
  print(logits)
  print("Printing labels")
  print(labels)
  
  
  

  #if mode in (tf.estimator.ModeKeys.PREDICT, tf.estimator.ModeKeys.EVAL):#To Do

  if mode in (tf.estimator.ModeKeys.TRAIN, tf.estimator.ModeKeys.EVAL):
    global_step = tf.train.get_or_create_global_step()
    loss = loss=custom_loss(labels,logits)
    #loss = tf.keras.losses.mean_squared_error(labels,logits)
    tf.summary.scalar('Loss', loss)
    #oss = custom_loss(labels,logits)
    # tf.summary.scalar('Loss function', loss)

  if mode == tf.estimator.ModeKeys.PREDICT:
    predictions = {'coordinates': logits}
    export_outputs = {'predictions': tf.estimator.export.PredictOutput(predictions)}
    return tf.estimator.EstimatorSpec(mode, predictions=predictions, export_outputs=export_outputs)
  #return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001,beta1=0.9,beta2=0.999,epsilon=1e-08,use_locking=False,name='Adam')
    train_op = optimizer.minimize(loss, global_step=global_step)
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

  '''if mode == tf.estimator.ModeKeys.EVAL:
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(label_indices, predicted_indices)
    }
    return tf.estimator.EstimatorSpec(
        mode, loss=loss, eval_metric_ops=eval_metric_ops)'''
  

In [0]:
run_config = tf.estimator.RunConfig(save_checkpoints_steps=10, model_dir='./Graph',save_summary_steps=10)

estimator = tf.estimator.Estimator(model_fn=model_fn, config=run_config)

# There is another Exporter named FinalExporter

train_spec = tf.estimator.TrainSpec(input_fn= lambda:input_fn(files_pattern=input_pattern, batch_size = batch_size, mode=tf.estimator.ModeKeys.TRAIN))


# eval_spec = tf.estimator.EvalSpec(
#   input_fn=generate_input_fn(file_names=valid_data_files,
#                              mode=tf.estimator.ModeKeys.EVAL,
#                              batch_size=10)'
#   steps=FLAGS.eval_steps, exporters=exporter)

INFO:tensorflow:Using config: {'_model_dir': './Graph', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd55b96ab38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
writer.add_graph(tf.get_default_graph())
writer.flush()

In [0]:
estimator.train(lambda: input_fn(files_pattern=input_pattern,  batch_size= batch_size, mode=tf.estimator.ModeKeys.TRAIN), steps=100000)

INFO:tensorflow:Calling model_fn.
train
pre (?, 784)
post (?, 28, 28, 1)
Tensor("my_reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Conv_1/Relu:0", shape=(?, 24, 24, 16), dtype=float32)
After flatten
Tensor("Flatten_1/Reshape:0", shape=(?, 9216), dtype=float32)
Tensor("dense/Sigmoid:0", shape=(?, 5), dtype=float32)
Printing Logits
Tensor("dense/Sigmoid:0", shape=(?, 5), dtype=float32)
Printing labels
Tensor("IteratorGetNext:1", shape=(?, 5), dtype=float32, device=/device:CPU:0)
Tensor("dense/Sigmoid:0", shape=(?, 5), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./Graph/model.ckpt.
INFO:tensorflow:loss = 0.52047306, step = 1
INFO:tensorflow:Saving checkpoints for 10 into ./Graph/model.ckpt.
INFO:tensorflow:Saving checkpoints for 20 into ./Graph/model.ckpt.
INFO:t

In [0]:
def input_fn_predict():
 
  dataset = tf.data.TFRecordDataset(['/gdrive/My Drive/indic2019/TFRecords/10_13.tfrecord'])
    # Transformation
  dataset = dataset.map(extract_fn)

  image, label = dataset.make_one_shot_iterator().get_next()
  features = {'images': image}
  
  
  return features